In [16]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [10]:
data = pd.read_csv('induce-data-2019-08-08.csv')
vocab = ['void',
         'C_E_F_T',
         'C_E_F_C',
         'C_E_F_O',
         'A_E_F_T',
         'A_E_F_O',
         'A_E_F_C',
         'G_E_F_C',
         'G_E_F_T',
         'G_E_F_O',
         'A_E_M_T',
         'A_E_M_O',
         'A_E_M_C',
         'G_E_M_O',
         'G_E_M_C',
         'G_E_M_T',
         'C_E_M_O',
         'C_E_M_C',
         'C_E_M_T',
         'C_H_F_CO',
         'C_H_F_CT',
         'C_H_F_OT',
         'G_H_F_OT',
         'G_H_F_CO',
         'G_H_F_CT',
         'A_H_F_CT',
         'A_H_F_OT',
         'A_H_F_CO',
         'C_H_M_CO',
         'C_H_M_CT',
         'C_H_M_OT',
         'A_H_M_CT',
         'A_H_M_OT',
         'A_H_M_CO',
         'G_H_M_OT',
         'G_H_M_CO',
         'G_H_M_CT',]
labels = ['correct',
          'wrong',
          'type',
          'orientation',
          'color']
def qts_to_seq(qts, vocab, labels):
    x = np.zeros(63)
    y = np.zeros(5)
    for i in range(len(qts)):
        x[i] = vocab.index(qts.iloc[i, 4])+labels.index(str(qts.iloc[i, 5]))
    y = labels.index(str(qts.iloc[len(qts)-1, 5]))
    return x, y
print(qts_to_seq(data.iloc[0:5, :], vocab, labels))
        

(array([1., 2., 3., 4., 5., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 0)


In [11]:
X = []
Y = []
users = list(dict.fromkeys(data.loc[:, "user"]))
for u in tqdm(users, desc='creating input matrix'):

        crt_usr_df = data[data.user == u]
        for i in range(len(crt_usr_df)):
            for j in range(2,63):
                x, y = qts_to_seq(crt_usr_df.iloc[0:j, :], vocab, labels)
                X.append(x)
                Y.append(y)
print(np.array(X).shape)
print(np.array(Y).shape)



creating input matrix: 100%|██████████| 222/222 [08:19<00:00,  2.07s/it]


(851987, 63)
(851987,)


In [ ]:
from keras import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
X = np.array(X)
Y= np.array(Y)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)
model = Sequential()
model.add(Embedding(len(vocab), 63, input_length=63))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 63, 63)            2331      
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               65600     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 68,032
Trainable params: 68,032
Non-trainable params: 0
_________________________________________________________________
None
Instructions for updating:
Use tf.cast instead.
Epoch 1/3
 91200/851987 [==>...........................] - ETA: 36:11 - loss: -18.4476 - acc: 0.0423